In [5]:
import pyomo
import pyomo.opt
import pyomo.environ as pe
import pandas
import itertools
import datetime

def DEROPTMODEL(der_data, schedule):

    SubStation_keys = ['SubStationID', 'SubTimeStamp', 'LoadForecast', 'PriceForecast', 'ImportMax']
    SelfScheduled_keys = ['SelfScheduledID', 'SSTimeStamp', 'SSGen', 'SSGenPrice']
    Battery_keys = ['BatteryID', 'BatTimeStamp', 'SocMin', 'SocMax', 'SocInit', 'SocTerm', 'ChargeRateMin', 'ChargeRateMax', 'DischargeRateMin', 'DischargeRateMax', 'BatGenPrice', 'BatChargePrice','Efficiency', 'BatGenCommitInit', 'BatGenCommitTerm', 'LoadCommitInit', 'LoadCommitTerm']
    DemandResponse_keys = ['DRID', 'DRTimeStamp', 'DownTimeMin','UpTimeMax','LoadReductionMin','LoadReductionMax','DRPrice']
    Dispatchable_keys = ['DispatchableID', 'DisTimeStamp', 'DisGenMin', 'DisGenMax', 'DisGenPrice', 'RampUpMax', 'RampDownMax', 'DisGenCommitInit', 'DisGenCommitTerm']

    unit_time = float(schedule['Horizon(Hour)'].tolist()[0])*float(schedule['Resolution(Fraction)'].tolist()[0])

    #### Prepare data source
    SubStation_df = der_data[SubStation_keys]
    SubStation_df = SubStation_df.dropna()
    SelfScheduled_df = der_data[SelfScheduled_keys]
    SelfScheduled_df = SelfScheduled_df.dropna()
    Battery_df = der_data[Battery_keys]
    Battery_df = Battery_df.dropna()
    DemandResponse_df = der_data[DemandResponse_keys]
    DemandResponse_df = DemandResponse_df.dropna()
    Dispatchable_df = der_data[Dispatchable_keys]
    Dispatchable_df = Dispatchable_df.dropna()

    # get joint keys between resource and time stamp
    TimeStamp_list = sorted(set(SubStation_df['SubTimeStamp'].tolist()))
    SubStation_list=sorted(set(SubStation_df['SubStationID'].tolist()))
    SubStation_varkeylist = list(itertools.product(SubStation_list,TimeStamp_list))
    SelfScheduled_list=sorted(set(SelfScheduled_df['SelfScheduledID'].tolist()))
    SelfScheduled_varkeylist = list(itertools.product(SelfScheduled_list,TimeStamp_list))       
    Battery_list=sorted(set(Battery_df['BatteryID'].tolist()))
    Battery_varkeylist = list(itertools.product(Battery_list,TimeStamp_list))       
    DemandResponse_list=sorted(set(DemandResponse_df['DRID'].tolist()))
    DemandResponse_varkeylist = list(itertools.product(DemandResponse_list,TimeStamp_list))       
    Dispatchable_list=sorted(set(Dispatchable_df['DispatchableID'].tolist()))
    Dispatchable_varkeylist = list(itertools.product(Dispatchable_list,TimeStamp_list))

    # get indices of data frames ready
    SubStation_df.set_index(['SubStationID','SubTimeStamp'], inplace=True)
    Battery_df.set_index(['BatteryID','BatTimeStamp'], inplace=True)
    SelfScheduled_df.set_index(['SelfScheduledID','SSTimeStamp'], inplace=True)
    DemandResponse_df.set_index(['DRID','DRTimeStamp'], inplace=True)
    Dispatchable_df.set_index(['DispatchableID','DisTimeStamp'], inplace=True)
    
    #### create model
    dermodel = pe.ConcreteModel()

    # declare sets
    dermodel.substation_set =pe.Set(initialize=SubStation_varkeylist)
    dermodel.selfscheduled_set =pe.Set(initialize=SelfScheduled_varkeylist)
    dermodel.battery_set =pe.Set(initialize=Battery_varkeylist)
    dermodel.demandresponse_set =pe.Set(initialize=DemandResponse_varkeylist)
    dermodel.dispatchable_set =pe.Set(initialize=Dispatchable_varkeylist)
    
    # declare variables
    dermodel.substation_load = pe.Var(dermodel.substation_set, domain=pe.NonNegativeReals)
    dermodel.substation_sale = pe.Var(dermodel.substation_set, domain=pe.NonNegativeReals)
    dermodel.substation_violation = pe.Var(dermodel.substation_set, domain=pe.NonNegativeReals)
    
    dermodel.battery_gen = pe.Var(dermodel.battery_set, domain=pe.NonNegativeReals)
    dermodel.battery_charge = pe.Var(dermodel.battery_set, domain=pe.NonNegativeReals)
    dermodel.battery_soc = pe.Var(dermodel.battery_set, domain=pe.NonNegativeReals)
    dermodel.battery_gencommit = pe.Var(dermodel.battery_set, domain=pe.Binary)
    dermodel.battery_chargecommit = pe.Var(dermodel.battery_set, domain=pe.Binary)

    dermodel.load_reduction = pe.Var(dermodel.demandresponse_set, domain=pe.NonNegativeReals)
    dermodel.load_reductioncommit = pe.Var(dermodel.demandresponse_set, domain=pe.Binary)
    dermodel.startup = pe.Var(dermodel.demandresponse_set, domain=pe.Binary)
    dermodel.shutdown = pe.Var(dermodel.demandresponse_set, domain=pe.Binary)

    dermodel.self_scheduled = pe.Var(dermodel.selfscheduled_set, domain=pe.NonNegativeReals)

    dermodel.dispatchable = pe.Var(dermodel.dispatchable_set, domain=pe.NonNegativeReals)
    dermodel.dispatchable_commit = pe.Var(dermodel.dispatchable_set, domain=pe.Binary)
    dermodel.dispatchable_startup = pe.Var(dermodel.dispatchable_set, domain=pe.Binary)
    dermodel.dispatchable_shutdown = pe.Var(dermodel.dispatchable_set, domain=pe.Binary)
    
    #### declare objective
    substation_obj_str = 'sum([dermodel.substation_load[x]*SubStation_df.ix[x,\'PriceForecast\']*unit_time for x in SubStation_varkeylist])'
    battery_pos_obj_str = 'sum([dermodel.battery_gen[x]*Battery_df.ix[x,\'BatGenPrice\']*unit_time for x in Battery_varkeylist])'
    battery_neg_obj_str = 'sum([dermodel.battery_charge[x]*Battery_df.ix[x,\'BatChargePrice\']*unit_time for x in Battery_varkeylist])'
    dr_obj_str = 'sum([dermodel.load_reduction[x]*DemandResponse_df.ix[x,\'DRPrice\']*unit_time for x in DemandResponse_varkeylist])'
    ss_obj_str = 'sum([dermodel.self_scheduled[x]*SelfScheduled_df.ix[x,\'SSGenPrice\']*unit_time for x in SelfScheduled_varkeylist])'
    dis_obj_str = 'sum([dermodel.dispatchable[x]*Dispatchable_df.ix[x,\'DisGenPrice\']*unit_time for x in Dispatchable_varkeylist])'
    obj_str = 'dermodel.obj = pe.Objective(expr=' +substation_obj_str+'+'+battery_pos_obj_str+'-'+battery_neg_obj_str+'+'+dr_obj_str+'+'+ss_obj_str+'+'+dis_obj_str +')'
    exec(obj_str)
    
    #### declare constraints

    # power balance constraints
    for t in TimeStamp_list:
        substation_load_str = 'sum([dermodel.substation_load[(x,t)] for x in SubStation_list])'
        battery_load_str = 'sum([dermodel.battery_gen[(x,t)] for x in Battery_list])'        
        ss_load_str = 'sum([dermodel.self_scheduled[(x,t)] for x in SelfScheduled_list])'
        dis_load_str = 'sum([dermodel.dispatchable[(x,t)] for x in Dispatchable_list])'
        forecast_str = 'sum([SubStation_df.ix[(x,t),\'LoadForecast\'] for x in SubStation_list])'
        substation_sale_str = 'sum([dermodel.substation_sale[(x,t)] for x in SubStation_list])'
        battery_charge_str = 'sum([dermodel.battery_charge[(x,t)] for x in Battery_list])'        
        DR_str = 'sum([dermodel.load_reduction[(x,t)] for x in DemandResponse_list])' 
        sum_str = substation_load_str+'+'+battery_load_str+'+'+ss_load_str+'+'+dis_load_str+'=='+forecast_str+'+'+substation_sale_str+'+'+battery_charge_str+'-'+DR_str
        const_str = 'dermodel.balance_const_'+t.replace(' ','_').replace(':','_').replace('-','_') +'=pe.Constraint(expr='+sum_str+')'
        exec(const_str)

    # substation Constraints
    for x in SubStation_varkeylist:
        battery_violation_const_str = 'dermodel.substation_load[x]<=SubStation_df.ix[x,\'ImportMax\']+dermodel.substation_violation[x]'
        const_str = 'dermodel.violation_const_'+x[0]+'_'+x[1].replace(' ','_').replace(':','_').replace('-','_')+'=pe.Constraint(expr='+battery_violation_const_str +')'
        exec(const_str)

    # self-scheduled constraints
    for x in SelfScheduled_varkeylist:
        ss_const_str = 'dermodel.self_scheduled[x]==SelfScheduled_df.ix[x,\'SSGen\']'
        const_str = 'dermodel.ss_const_'+x[0]+'_'+x[1].replace(' ','_').replace(':','_').replace('-','_')+'=pe.Constraint(expr='+ss_const_str+')'
        exec(const_str)

    # battery constraints
    # 1. state of charge constraints
    for x in Battery_varkeylist:
        bat_soc_const_low_str = 'dermodel.bat_socmin_'+x[0]+'_'+x[1].replace(' ','_').replace(':','_').replace('-','_')+'=pe.Constraint(expr=dermodel.battery_soc[x] >= Battery_df.ix[x,\'SocMin\'])'
        exec(bat_soc_const_low_str)
        bat_soc_const_high_str = 'dermodel.bat_socmax_'+x[0]+'_'+x[1].replace(' ','_').replace(':','_').replace('-','_')+'=pe.Constraint(expr=dermodel.battery_soc[x] <= Battery_df.ix[x,\'SocMax\'])'
        exec(bat_soc_const_high_str)
    # 2. no charge and discharge(gen) at the same time
    for x in Battery_varkeylist:
        charge_gen_const_str = 'dermodel.bat_charge_gen_'+x[0]+'_'+x[1].replace(' ','_').replace(':','_').replace('-','_')+'=pe.Constraint(expr=dermodel.battery_chargecommit[x] + dermodel.battery_gencommit[x] <=1)'
        exec(charge_gen_const_str)
    # 3. generation limits
    for x in Battery_varkeylist:
        gen_limit_const_str = 'dermodel.bat_gen_limit_low_'+x[0]+'_'+x[1].replace(' ','_').replace(':','_').replace('-','_')+'=pe.Constraint(expr=dermodel.battery_gen[x] >= Battery_df.ix[x,\'DischargeRateMin\']*dermodel.battery_gencommit[x])'
        exec(gen_limit_const_str)
        gen_limit_const_str = 'dermodel.bat_gen_limit_high_'+x[0]+'_'+x[1].replace(' ','_').replace(':','_').replace('-','_')+'=pe.Constraint(expr=dermodel.battery_gen[x] <= Battery_df.ix[x,\'DischargeRateMax\']*dermodel.battery_gencommit[x])'
        exec(gen_limit_const_str)
    # 4. charge limits
    for x in Battery_varkeylist:
        charge_limit_const_str = 'dermodel.bat_charge_limit_low_'+x[0]+'_'+x[1].replace(' ','_').replace(':','_').replace('-','_')+'=pe.Constraint(expr=dermodel.battery_charge[x] >= Battery_df.ix[x,\'ChargeRateMin\']*dermodel.battery_chargecommit[x])'
        exec(charge_limit_const_str)
        charge_limit_const_str = 'dermodel.bat_charge_limit_high_'+x[0]+'_'+x[1].replace(' ','_').replace(':','_').replace('-','_')+'=pe.Constraint(expr=dermodel.battery_charge[x] <= Battery_df.ix[x,\'ChargeRateMax\']*dermodel.battery_chargecommit[x])'
        exec(charge_limit_const_str)
    # 5. Battery init and term conditions
    for b in Battery_list:
        initkey = (b,TimeStamp_list[0])
        #termkey = (b,TimeStamp_list[-1])
        bat_init_const_str = 'dermodel.bat_init_'+b+'=pe.Constraint(expr=dermodel.battery_soc[initkey] == Battery_df.ix[initkey,\'SocInit\'])'
        exec(bat_init_const_str)
        #bat_term_const_str = 'dermodel.bat_term_'+b+'=pe.Constraint(expr=dermodel.battery_soc[termkey] == Battery_df.ix[termkey,\'SocTerm\'])'
        #exec(bat_term_const_str)
    # 6. dynamic retriction
    for b in Battery_list:
        for t in range(1,len(TimeStamp_list)):
            first_key = (b,TimeStamp_list[t-1])
            second_key = (b,TimeStamp_list[t])
            bat_dynamic_const_str = 'dermodel.bat_dynamic_'+b + '_' + str(t)+'=pe.Constraint(expr=dermodel.battery_soc[second_key] == dermodel.battery_soc[first_key]+(dermodel.battery_charge[second_key]*Battery_df.ix[second_key,\'Efficiency\']-dermodel.battery_gen[second_key])*unit_time)'
            exec(bat_dynamic_const_str)
            
    # dispatchable constraints
    # 1. no startup and shutdown at the same time
    for x in Dispatchable_varkeylist:
        dis_start_shut_sametime_const_str = 'dermodel.dis_start_shut_same_'+x[0]+'_'+x[1].replace(' ','_').replace(':','_').replace('-','_')+'=pe.Constraint(expr=dermodel.dispatchable_startup[x] + dermodel.dispatchable_shutdown[x] <=1)'
        exec(dis_start_shut_sametime_const_str)

    # 2. Startup and shutdown 
    for d in Dispatchable_list:
        for t in range(1,len(TimeStamp_list)):
            first_key = (d,TimeStamp_list[t-1])
            second_key = (d,TimeStamp_list[t])
            dis_start_shut_const_str = 'dermodel.dis_start_shut_'+d + '_' + str(t)+'=pe.Constraint(expr=dermodel.dispatchable_startup[second_key]- dermodel.dispatchable_shutdown[second_key] == dermodel.dispatchable_commit[second_key]-dermodel.dispatchable_commit[first_key])'
            exec(dis_start_shut_const_str)

    # 3. init and term conditions
    for d in Dispatchable_list:
        initkey = (d,TimeStamp_list[0])
        #termkey = (d,TimeStamp_list[-1])
        dis_init_const_str = 'dermodel.dis_init_'+d+'=pe.Constraint(expr=dermodel.dispatchable_commit[initkey] == Dispatchable_df.ix[initkey,\'DisGenCommitInit\'])'
        exec(dis_init_const_str)
        #dis_term_const_str = 'dermodel.dis_term_'+d+'=pe.Constraint(expr=dermodel.dispatchable_commit[termkey] == Dispatchable_df.ix[termkey,\'DisGenCommitTerm\'])'
        #exec(dis_term_const_str)

    # 4. Ramping up dynamic
    for d in Dispatchable_list:
        for t in range(1,len(TimeStamp_list)):
            first_key = (d,TimeStamp_list[t-1])
            second_key = (d,TimeStamp_list[t])
            ramp_up_str ='dermodel.dispatchable[second_key]-dermodel.dispatchable[first_key]<=(1-dermodel.dispatchable_startup[second_key])*Dispatchable_df.ix[second_key,\'RampUpMax\']*unit_time+Dispatchable_df.ix[second_key,\'DisGenMin\']*dermodel.dispatchable_startup[second_key]'
            ramp_up_const_str = 'dermodel.dis_ramp_up_'+d + '_' + str(t)+'=pe.Constraint(expr=' +ramp_up_str+ ')'
            exec(ramp_up_const_str)

    # 5. Ramping down dynamic
    for d in Dispatchable_list:
        for t in range(1,len(TimeStamp_list)):
            first_key = (d,TimeStamp_list[t-1])
            second_key = (d,TimeStamp_list[t])
            ramp_down_str ='dermodel.dispatchable[second_key]-dermodel.dispatchable[first_key]>=-(1-dermodel.dispatchable_shutdown[second_key])*Dispatchable_df.ix[second_key,\'RampDownMax\']*unit_time+Dispatchable_df.ix[second_key,\'DisGenMax\']*dermodel.dispatchable_shutdown[second_key]'
            ramp_down_const_str = 'dermodel.dis_ramp_down_'+d + '_' + str(t)+'=pe.Constraint(expr=' +ramp_down_str+ ')'
            exec(ramp_down_const_str)
            
    # demand response constraints
    # 1. load reduction bound constraint
    for x in DemandResponse_varkeylist:
        dr_limit_const_str = 'dermodel.dr_gen_limit_low_'+x[0]+'_'+x[1].replace(' ','_').replace(':','_').replace('-','_')+'=pe.Constraint(expr=dermodel.load_reduction[x] >= DemandResponse_df.ix[x,\'LoadReductionMin\']*dermodel.load_reductioncommit[x])'
        exec(dr_limit_const_str)
        dr_limit_const_str = 'dermodel.dr_gen_limit_high_'+x[0]+'_'+x[1].replace(' ','_').replace(':','_').replace('-','_')+'=pe.Constraint(expr=dermodel.load_reduction[x] <= DemandResponse_df.ix[x,\'LoadReductionMax\']*dermodel.load_reductioncommit[x])'
        exec(dr_limit_const_str)

    # 2. no startup and shutdown at the same time
    for x in DemandResponse_varkeylist:
        dr_start_shut_sametime_const_str = 'dermodel.dr_start_shut_same_'+x[0]+'_'+x[1].replace(' ','_').replace(':','_').replace('-','_')+'=pe.Constraint(expr=dermodel.startup[x] + dermodel.shutdown[x] <=1)'
        exec(dr_start_shut_sametime_const_str)
        
    # 3. Startup and shutdown 
    for d in DemandResponse_list:
        for t in range(1,len(TimeStamp_list)):
            first_key = (d,TimeStamp_list[t-1])
            second_key = (d,TimeStamp_list[t])
            dr_start_shut_const_str = 'dermodel.dr_start_shut_'+d + '_' + str(t)+'=pe.Constraint(expr=dermodel.startup[second_key]- dermodel.shutdown[second_key] == dermodel.load_reductioncommit[second_key]-dermodel.load_reductioncommit[first_key])'
            exec(dr_start_shut_const_str)

    # 4. max up time
    for d in DemandResponse_list:
        for t in range(1,len(TimeStamp_list)):
            tstart = max([t+1-DemandResponse_df.ix[(d,TimeStamp_list[t]),'UpTimeMax'],0])
            uprange = TimeStamp_list[tstart:t+1]
            max_up_time_str = 'dermodel.dr_up_max_'+d + '_' + str(t)+'=pe.Constraint(expr=sum([dermodel.load_reductioncommit[d,x] for x in uprange])<=DemandResponse_df.ix[(d,uprange[0]),\'UpTimeMax\'])'
            exec(max_up_time_str)

    # 5. min down time
    for d in DemandResponse_list:
        for t in range(1,len(TimeStamp_list)):
            tstart = max([t+1-DemandResponse_df.ix[(d,TimeStamp_list[t]),'DownTimeMin'],0])
            downrange = TimeStamp_list[tstart:t+1]
            min_down_time_str = 'dermodel.dr_down_min_'+d + '_' + str(t)+'=pe.Constraint(expr=sum([dermodel.startup[d,x] for x in downrange])<=1-DemandResponse_df.ix[(d,downrange[0]),\'DownTimeMin\'])'
            exec(min_down_time_str)
            
    #### get the solver ####
    ## Change here for a different solver!
    solver = pyomo.opt.SolverFactory('cbc',executable=".\\solverdir\\cbc\\win\\64\\cbc.exe")
    
    #### run
    results = solver.solve(dermodel)
    results.write()
    print "computation complete!"
    
    #### assemble results
    print '---'
    objval = float(dermodel.obj())

    id_list = []
    timestamp_list=[]
    quantity_name_list = []
    quantity_val_list = []
    objval_list = []
    computeStamp_list =[]
    
    result_df = pandas.DataFrame(columns=['ID','Time Stamp','Quantity','Value','ObjValue','computeStamp'])
    date_time_base = datetime.datetime(2017,1,1,0,0,0)
    date_time_now = (datetime.datetime.now()-date_time_base).total_seconds()
    
    for k in sorted(dermodel.substation_load):
        id_list.append(k[0]) 
        timestamp_list.append(k[1])
        quantity_name_list.append('load')
        quantity_val_list.append(float(dermodel.substation_load[k].value))
        objval_list.append(objval)
        computeStamp_list.append(date_time_now)

    for k in sorted(dermodel.substation_sale):
        id_list.append(k[0]) 
        timestamp_list.append(k[1])
        quantity_name_list.append('sale')
        quantity_val_list.append(float(dermodel.substation_sale[k].value))
        objval_list.append(objval)
        computeStamp_list.append(date_time_now)
        
    for k in sorted(dermodel.battery_gen):
        id_list.append(k[0]) 
        timestamp_list.append(k[1])
        quantity_name_list.append('discharge')
        quantity_val_list.append(float(dermodel.battery_gen[k].value*dermodel.battery_gencommit[k].value))
        objval_list.append(objval)
        computeStamp_list.append(date_time_now)
        
    for k in sorted(dermodel.battery_charge):
        id_list.append(k[0]) 
        timestamp_list.append(k[1])
        quantity_name_list.append('charge')
        quantity_val_list.append(float(dermodel.battery_charge[k].value*dermodel.battery_chargecommit[k].value))
        objval_list.append(objval)
        computeStamp_list.append(date_time_now)
        
    for k in sorted(dermodel.load_reduction):
        id_list.append(k[0]) 
        timestamp_list.append(k[1])
        quantity_name_list.append('load reduction')
        quantity_val_list.append(float(dermodel.load_reduction[k].value*dermodel.load_reductioncommit[k].value))
        objval_list.append(objval)
        computeStamp_list.append(date_time_now)
        
    for k in sorted(dermodel.dispatchable):
        id_list.append(k[0]) 
        timestamp_list.append(k[1])
        quantity_name_list.append('generation')
        quantity_val_list.append(float(dermodel.dispatchable[k].value*dermodel.dispatchable_commit[k].value))
        objval_list.append(objval)
        computeStamp_list.append(date_time_now)

    result_df['ID']=id_list
    result_df['Time Stamp']=timestamp_list
    result_df['Quantity']=quantity_name_list
    result_df['Value']=quantity_val_list
    result_df['ObjValue']=objval_list
    result_df['computeStamp']=computeStamp_list
    
    print 'success!'

    return result_df

## entry point

der_data = pandas.read_csv('deroptinput.csv')
schedule_df = pandas.read_csv('schedule.csv')

result_df = DEROPTMODEL(der_data,schedule_df)
print result_df

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmpsgvzmc.pyomo
  Lower bound: 26609.3352523
  Upper bound: 26609.3352523
  Number of objectives: 1
  Number of constraints: 180
  Number of variables: 157
  Number of nonzeros: 406
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  Termination condition: optimal
  Error rc: 0
  Time: 0.0269999504089
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of so